In [ ]:
# ALGORITHM 2

%run utilities.ipynb

def _algorithm2(gens, m0):
    """EF_{m0} checker with aggressive early exits and caching."""
    G = GL(2, _R(m0)).subgroup(gens)
    
    def check_EFm0_pair_from_Gm0(Gm0, m0, a, b):
        """EF_{m0} check with aggressive early exits."""
        a = Integer(a); b = Integer(b); m0 = Integer(m0)
    
        if gcd(a, b) != 1 or m0 % (a*b) != 0:
            return False  # Early exit for invalid pairs
    
        # Always enumerate elements for kernel computation (required for correctness)
        elems_m0 = list(Gm0)
        A_elems = _unique_reductions(elems_m0, a)
        B_elems = _unique_reductions(elems_m0, b)
        AB_elems = _unique_reductions(elems_m0, a*b)
        
        # Compute kernels
        IdA = identity_matrix(_R(a), 2)
        IdB = identity_matrix(_R(b), 2)
        
        NA = {}
        NB = {}
        for gab in AB_elems:
            ga = _reduce_mod_cached(gab, a)
            gb = _reduce_mod_cached(gab, b)
            if gb == IdB:
                NA[_mat_key(ga)] = ga
            if ga == IdA:
                NB[_mat_key(gb)] = gb
        
        NA_elems = list(NA.values())
        NB_elems = list(NB.values())
        
        if not NA_elems or not NB_elems:
            return False  # Early exit
        
        # Quick size check
        ordA = len(A_elems)
        ordB = len(B_elems)
        ordNA = len(NA_elems)
        ordNB = len(NB_elems)
        
        if ordA % ordNA != 0 or ordB % ordNB != 0:
            return False  # Early exit
        
        QA = ordA // ordNA
        QB = ordB // ordNB
        if QA != QB:
            return False  # Early exit
        
        # Primitive vectors and orbit reps - use cached versions
        Va_prim = primitive_vectors_exact_order(a)
        Vb_prim = primitive_vectors_exact_order(b)
        
        # Use reduced generator sets for orbit computations
        gens_m0 = list(Gm0.gens())
        A_gens = [_reduce_mod_cached(g, a) for g in gens_m0]
        B_gens = [_reduce_mod_cached(g, b) for g in gens_m0]
        
        repsA = orbit_reps_on_vectors(Va_prim, A_gens)
        repsB = orbit_reps_on_vectors(Vb_prim, B_gens)
        
        # EF_{m0} test with aggressive early exit on first failure
        # Check smaller set first for potential early exit
        if len(repsA) <= len(repsB):
            for v in repsA:
                SA_elems = stabilizer_elements_from_element_list(A_elems, v)
                ord_genA = generated_subgroup_order(SA_elems + NA_elems)
                if ord_genA != ordA:
                    return False  # Early exit
            
            for v in repsB:
                SB_elems = stabilizer_elements_from_element_list(B_elems, v)
                ord_genB = generated_subgroup_order(SB_elems + NB_elems)
                if ord_genB != ordB:
                    return False  # Early exit
        else:
            # Check B first if it's smaller
            for v in repsB:
                SB_elems = stabilizer_elements_from_element_list(B_elems, v)
                ord_genB = generated_subgroup_order(SB_elems + NB_elems)
                if ord_genB != ordB:
                    return False  # Early exit
            
            for v in repsA:
                SA_elems = stabilizer_elements_from_element_list(A_elems, v)
                ord_genA = generated_subgroup_order(SA_elems + NA_elems)
                if ord_genA != ordA:
                    return False  # Early exit
    
        return True
    
    # Driver for all coprime pairs
    def EFm0_checks_from_Gm0(Gm0, m0, mode="ppairs"):
        """Driver with minimal pair checking and early exits."""
        m0 = Integer(m0)
        
        # Use ppairs mode by default (much faster)
        if mode == "ppairs":
            pp = []
            for p, e in m0.factor():
                for k in range(1, e+1):
                    pp.append(Integer(p**k))
            pp = sorted(set(pp))
            
            pairs = []
            for i, a in enumerate(pp):
                for b in pp[i:]:
                    if a > 1 and b > 1 and gcd(a, b) == 1 and (m0 % (a*b) == 0):
                        pairs.append((a, b))
        else:
            # Fallback to all pairs if needed
            divs = [Integer(d) for d in m0.divisors()]
            pairs = []
            for a in divs:
                if a <= 1:
                    continue
                for b in divs:
                    if b < a or b <= 1:
                        continue
                    if gcd(a, b) == 1 and (m0 % (a*b) == 0):
                        pairs.append((a, b))
        
        # Sort pairs by product size to check smaller products first
        pairs.sort(key=lambda x: x[0] * x[1])
        
        # Check pairs with early exit
        for a, b in pairs:
            if not check_EFm0_pair_from_Gm0(Gm0, m0, a, b):
                return False  # Early exit on first failure
        
        return True
    
    return EFm0_checks_from_Gm0(G, m0, mode="ppairs")